In [1]:
from pyomo.environ import * 
from pyomo.dae import *
from pyomo.contrib.sensitivity_toolbox.sens import get_dsdp


In [14]:
model = ConcreteModel()
#model.time = ContinuousSet(bounds=(0,10))
#model.X = Var(model.time)

model.A_b = Param(initialize=2, mutable=True)
model.component_list = Set(initialize=["N2", "CO2", "H2O"], doc="List of components")

z_bounds = (0, 1)
# z_init_points = tuple(np.linspace(0.0,0.01,5))+tuple(np.linspace(0.99,1,5))
# z_init_points = tuple(np.linspace(0,0.1,5)) + (0.99,)
z_init_points = (0.01, 0.99)

o_bounds = (0, 1)
# o_init_points = tuple(np.linspace(0.0,0.01,5))+tuple(np.linspace(0.99,1,5))
# o_init_points = tuple(np.linspace(0,0.1,5)) + (0.99,)
o_init_points = (0.01, 0.99)

model.z = ContinuousSet(
    doc="axial nodes [dimensionless]",
    bounds=z_bounds,
    initialize=z_init_points,
)

model.o = ContinuousSet(
    doc="adsorption theta nodes [dimensionless]",
    bounds=o_bounds,
    initialize=o_init_points,
)

model.a = Var(model.z, model.o, initialize=1, within=Reals)

#model.A_b = 2 

model.Flux_kzo = Var(
    model.component_list,
    model.z,
    model.o,
    bounds=(1, 50),
    units=units.mol / units.m**2 / units.s,
    doc="Gas phse component flux [mol/m^2/s]",
)

model.dFluxdz = DerivativeVar(
    model.Flux_kzo,
    wrt=model.z,
    doc="axial derivative of component flux [mol/m^2 bed/s/dimensionless bed length]",
)


@model.Integral(
    model.component_list, 
    model.z,
    model.o,
    wrt=model.o,
    doc="Flow [mol/s]"
)
def Flow_kz(m,k,z,o):
    return m.Flux_kzo[k,z,o]*m.A_b


#@m.Expression(m.z, m.o, doc="[mol/s]")
def Flux_zo(m,z,o):
    return  m.a[z,o] == sum(m.Flux_kzo[k,z,o] for k in m.component_list)

model.con = Constraint(model.z, model.o, rule=Flux_zo)

#model.intX = Integral(model.time, wrt=model.time, rule=_intX)

#def _con1(m):
#    return m.a == m.intX + m.scale 
#model.con = Constraint(rule=_con1)

def _obj(m):
    return m.a
#model.obj = Objective(rule=_obj)

model.Obj = Objective(expr=0, sense=minimize)

In [15]:
discretizer = TransformationFactory('dae.collocation')
discretizer.apply_to(model,nfe=20,wrt=model.z,scheme='LAGRANGE-RADAU', ncp=3)
discretizer.apply_to(model,nfe=20,wrt=model.o,scheme='LAGRANGE-RADAU', ncp=3)

In [16]:
solver = SolverFactory("ipopt")
solver.solve(model, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 14701, 'Number of variables': 25864, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.27449941635131836}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [11]:
print(value(model.intX))

-0.6337995648274556


In [17]:
### run get_dsdp
param = {"A_b": 2}
param_list = ["A_b"]

dsdp, col = get_dsdp(model, param_list, param, tee=True)

W[K_AUG]...	[K_AUG_ASL]No n_rhs declared.
W[K_AUG]...	[K_AUG_ASL]Target log10mu:= -11.
[K_AUG] 0.1.0, Part of the IDAES PSE framework
Please visit https://idaes.org/
: dsdp_mode=
dsdp_mode=
W[K_AUG_ASL]...	[K_AUG_ASL]No ipopt_zL_in suffix declared, setting zL = 0.
W[K_AUG_ASL]...	[K_AUG_ASL]No ipopt_zU_in suffix declared, setting zU = 0.
W[K_AUG]...	[K_AUG_ASL]No f_timestamp suffix declared, Fallback to default writing mode.
I[K_AUG]...	[K_AUG_ASL] Filename for dot_sens dsdp_in_.in
W[K_AUG]...	[K_AUG_ASL]dsdp for linear C(x) + I*p = 0 override.
W[K_AUG]...	[INERTIA_STRATEGY]Wrong inertia(neig < m).
W[K_AUG]...	[INERTIA_STRATEGY]Attempting to make i_pert->d_c > 0.
W[K_AUG]...	[INERTIA_STRATEGY]Wrong inertia(neig < m).
W[K_AUG]...	[INERTIA_STRATEGY]d_c is already > 0
The Jacobian might be singular. Asking for better accuracy..
W[K_AUG]...	[INERTIA_STRATEGY]Wrong inertia(neig < m).
W[K_AUG]...	[INERTIA_STRATEGY]d_c is already > 0
The Jacobian might be singular. Asking for better accuracy.